# Phase 1: Extracting Triplets from Informal Proofs

Add src to the Python Path in the Notebook

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os


# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("../../..")))

## 1. Read Informal Proves
Read the LaTeX file. This file contains the informal proofs of the theorems in the book.

In [6]:
from IPython.display import display, Math, Latex
import re
from src.utils.file_utils import read_proof

# Load LaTeX proof
proof_latex = read_proof("../../data/proofs/persian/problem3/proof2.tex")

# Find the start and end positions
start = proof_latex.find(r"\begin{document}") + len(r"\begin{document}")
end = proof_latex.find(r"\end{document}")

# Extract the content between \begin{document} and \end{document}
informal_proof = proof_latex[start:end].strip()


# Replace any \section{...} with ## ...
informal_proof = re.sub(r"\\section\{([^}]+)\}", r"## \1", informal_proof)

# Display the LaTeX content
display(Latex(informal_proof))

<IPython.core.display.Latex object>

## 2. Extract Triplet proofs from Informal Proofs

In [7]:
from src.phase1.extract_triplets import extract_triplets

# Extract triplets
triplet = extract_triplets(informal_proof)
triplet

Triplet(entities=[Entity(id='1', name='tan a', label='Tangent of angle a', type='Function'), Entity(id='2', name='sin a', label='Sine of angle a', type='Function'), Entity(id='3', name='cos a', label='Cosine of angle a', type='Function'), Entity(id='4', name='1 - sin^2 a', label='Identity involving sine', type='Expression'), Entity(id='5', name='cos^2 a', label='Square of cosine of angle a', type='Expression'), Entity(id='6', name='(1 + sin a)(1 - sin a)', label='Product of expressions', type='Expression'), Entity(id='7', name='(1/cos a + tan a)(1 - sin a)', label='Left-hand side of the equation', type='Expression'), Entity(id='8', name='cos a', label='Right-hand side of the equation', type='Expression')], relations=[Relation(source='1', target='2', type='explains', name='Tangent definition'), Relation(source='2', target='3', type='explains', name='Sine and cosine relationship'), Relation(source='4', target='5', type='identity', name='Pythagorean identity'), Relation(source='6', target

## 3. Store Triplets into Neo4J DB

In [8]:
from src.utils.neo4j_utils import Neo4JUtils

# Initialize Neo4JUtils
neo4j = Neo4JUtils("bolt://localhost:7687", ("neo4j", "password"))

# Clean the database (delete all nodes and relationships)
neo4j.clean_database()

# Add nodes and relationships with step tracking
for entity in triplet.entities:
    neo4j.create_node(entity)  # Uses the current step (default is 0)
for relation in triplet.relations:
    neo4j.create_relation(relation)  # Uses the current step (default is 0)

# Increment the step counter for the next set of changes
neo4j.increment_step()

# Clean the database (delete nodes and relationships with step > 1)
neo4j.clean_database(step=1)

# Add or modify nodes and relationships in the next step
# Example:
# neo4j.create_node(new_entity)  # This will use the updated step counter (1)
# neo4j.create_relation(new_relation)  # This will use the updated step counter (1)

# Close the connection
neo4j.close()